In [8]:
import os
import cv2
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations.augmentations.transforms import ImageCompression
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.utils import to_categorical



# Define augmentation pipeline
augmentation_pipeline = A.Compose([
    A.Rotate(limit=15, p=0.5),  # Random rotation
    A.RandomSizedCrop(min_max_height=(100, 300), size=(640, 640), p=0.5),  # Corrected
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # Brightness and contrast adjustment
    A.OneOf([
        A.MotionBlur(blur_limit=3),  # Motion blur
        A.GaussianBlur(blur_limit=3),  # Gaussian blur
    ], p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Gaussian noise
    A.Perspective(scale=(0.05, 0.1), p=0.5),  # Perspective warp
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=4, p=0.5),  # Shadows
    A.ImageCompression(quality_lower=30, quality_upper=90, p=0.5),  # JPEG compression artifacts
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, min_holes=1, min_height=8, min_width=8, p=0.5),  # CutOut
])

# Paths
train_dir = r"D:\\WORK\\Baybayin App Project\\split_sampled_dataset_v1\\train"
output_dir = os.path.join(r"D:\\WORK\\Baybayin App Project\\split_sampled_dataset_v1\\augmented_train")
os.makedirs(output_dir, exist_ok=True)

# Enable GPU processing with TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is enabled for processing.")
else:
    print("GPU is not available. Falling back to CPU.")

# Apply augmentations for each class folder
print("Applying augmentations to training data...")
for class_name in tqdm(os.listdir(train_dir)):
    class_dir = os.path.join(train_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    class_output_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)

        # Skip non-image files
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Read image
        image = cv2.imread(image_path)
        if image is None:
            continue

        # Transfer image to TensorFlow tensor for potential GPU acceleration
        image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
        image_tensor = tf.expand_dims(image_tensor, axis=0)  # Add batch dimension

        # Augmentation must still use Albumentations (runs on CPU)
        augmented = augmentation_pipeline(image=image)
        augmented_image = augmented['image']

        # Convert back to numpy for saving
        augmented_image = tf.squeeze(image_tensor).numpy().astype('uint8')

        # Save augmented image
        output_path = os.path.join(class_output_dir, f"aug_{image_name}")
        cv2.imwrite(output_path, augmented_image)

print(f"Augmentation complete. Augmented images saved to: {output_dir}")


C:\Users\rojgi\AppData\Local\Temp\ipykernel_12616\1598499687.py:18: UserWarning: Argument 'var_limit' is not valid and will be ignored.
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Gaussian noise
C:\Users\rojgi\AppData\Local\Temp\ipykernel_12616\1598499687.py:20: UserWarning: Argument 'num_shadows_lower' is not valid and will be ignored.
  A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=4, p=0.5),  # Shadows
C:\Users\rojgi\AppData\Local\Temp\ipykernel_12616\1598499687.py:20: UserWarning: Argument 'num_shadows_upper' is not valid and will be ignored.
  A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=4, p=0.5),  # Shadows
C:\Users\rojgi\AppData\Local\Temp\ipykernel_12616\1598499687.py:21: UserWarning: Argument 'quality_lower' is not valid and will be ignored.
  A.ImageCompression(quality_lower=30, quality_upper=90, p=0.5),  # JPEG compression artifacts
C:\Users\rojgi\AppData\Loc

GPU is enabled for processing.
Applying augmentations to training data...


100%|██████████| 59/59 [24:45<00:00, 25.17s/it]

Augmentation complete. Augmented images saved to: D:\\WORK\\Baybayin App Project\\split_sampled_dataset_v1\\augmented_train


In [9]:

# Paths to dataset folders
train_dir = r"D:\WORK\Baybayin App Project\split_sampled_dataset_v1\augmented_train"
val_dir = r"D:\WORK\Baybayin App Project\split_sampled_dataset_v1\val"
test_dir = r"D:\WORK\Baybayin App Project\split_sampled_dataset_v1\test"

# Image dimensions and batch size
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 41300 images belonging to 59 classes.
Found 11800 images belonging to 59 classes.


In [10]:


# Define the objective function for Optuna
def objective(trial):
    model = Sequential()

    # Hyperparameters to tune
    num_conv_layers = trial.suggest_int("num_conv_layers", 1, 3)
    filter_count = trial.suggest_categorical("filter_count", [32, 64, 128])
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5])
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)

    # Build CNN with dynamic hyperparameters
    for _ in range(num_conv_layers):
        model.add(Conv2D(filter_count, (kernel_size, kernel_size), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
        model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(train_data.num_classes, activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

 # Track the best validation accuracy and corresponding epoch
    best_epoch = 0
    best_val_accuracy = 0.0

    for epoch in range(1, 31):  # Iterate through 1-30 epochs
        history = model.fit(
            train_data,
            epochs=1,
            validation_data=val_data,
            verbose=0
        )
        val_accuracy = history.history['val_accuracy'][0]

        # Update the best validation accuracy and epoch
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch

    # Log the best epoch for this trial
    trial.set_user_attr("best_epoch", best_epoch)
    return best_val_accuracy

# Run Optuna optimization with progress bar
class TQDMOptunaCallback(optuna.integration.TFKerasPruningCallback):
    def __init__(self, iterator):
        self.iterator = iterator

    def __call__(self, study, trial):
        self.iterator.update(1)


study = optuna.create_study(direction="maximize")
with tqdm(total=20) as pbar:
    study.optimize(lambda trial: objective(trial), n_trials=20, callbacks=[TQDMOptunaCallback(pbar)])

# Get the best hyperparameters
print("Best Hyperparameters:", study.best_params)
print("Best Epoch:", study.best_trial.user_attrs["best_epoch"])

# Build and train the final model with best hyperparameters
best_params = study.best_params

[I 2025-01-12 14:00:40,511] A new study created in memory with name: no-name-f189eb09-ed5a-4aa7-996c-47ce47fd6895
  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\rojgi\AppData\Local\Temp\ipykernel_12616\3556750993.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
[I 2025-01-12 14:45:55,681] Trial 0 finished with value: 0.8888135552406311 and parameters: {'num_conv_layers': 1, 'filter_count': 128, 'kernel_size': 5, 'dropout_rate': 0.2455854343480782, 'learning_rate': 0.0010252894210525947}. Best is trial 0 with value: 0.8888135552406311.
100%|██████████| 20/20 [8:24:40<00:00, 1514.01s/it]

Best Hyperparameters: {'num_conv_layers': 3, 'filter_count': 64, 'kernel_size': 3, 'dropout_rate': 0.37737044764714117, 'learning_rate': 0.0014360029060797065}
Best Epoch: 27


EPOCH NOT the OPTIMAL YET

In [13]:
final_model = Sequential()
for _ in range(best_params["num_conv_layers"]):
    final_model.add(Conv2D(best_params["filter_count"], (best_params["kernel_size"], best_params["kernel_size"]), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    final_model.add(MaxPooling2D((2, 2)))

final_model.add(Flatten())
final_model.add(Dropout(best_params["dropout_rate"]))
final_model.add(Dense(train_data.num_classes, activation='softmax'))

final_model.compile(optimizer=Adam(learning_rate=best_params["learning_rate"]), loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(
    train_data,
    epochs=study.best_trial.user_attrs["best_epoch"],
    validation_data=val_data
)

# Save the final model
final_model.save("version2Testing.h5")

# Evaluate on the test set
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

results = final_model.evaluate(test_data)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")


Epoch 1/27
1291/1291 [==============================] - 59s 45ms/step - loss: 1.0279 - accuracy: 0.7210 - val_loss: 0.4644 - val_accuracy: 0.8635
Epoch 2/27
1291/1291 [==============================] - 54s 42ms/step - loss: 0.4057 - accuracy: 0.8749 - val_loss: 0.3560 - val_accuracy: 0.8975
Epoch 3/27
1291/1291 [==============================] - 59s 46ms/step - loss: 0.2890 - accuracy: 0.9087 - val_loss: 0.2792 - val_accuracy: 0.9195
Epoch 4/27
1291/1291 [==============================] - 57s 44ms/step - loss: 0.2174 - accuracy: 0.9299 - val_loss: 0.2512 - val_accuracy: 0.9311
Epoch 5/27
1291/1291 [==============================] - 55s 42ms/step - loss: 0.1733 - accuracy: 0.9446 - val_loss: 0.2477 - val_accuracy: 0.9349
Epoch 6/27
1291/1291 [==============================] - 53s 41ms/step - loss: 0.1504 - accuracy: 0.9506 - val_loss: 0.2347 - val_accuracy: 0.9381
Epoch 7/27
1291/1291 [==============================] - 53s 41ms/step - loss: 0.1255 - accuracy: 0.9588 - val_loss: 0.2456 -

In [17]:


# Load test data
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Evaluate model
results = final_model.evaluate(test_data)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

# Predictions and true labels
y_true = test_data.classes
y_pred = final_model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

# One-hot encoding for AUC
y_true_one_hot = to_categorical(y_true, num_classes=len(test_data.class_indices))
auc = roc_auc_score(y_true_one_hot, y_pred, multi_class='ovr')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"AUC: {auc}")


Found 5900 images belonging to 59 classes.
185/185 [==============================] - 8s 40ms/step - loss: 0.3366 - accuracy: 0.9466
Test Loss: 0.33660224080085754, Test Accuracy: 0.946610152721405
185/185 [==============================] - 7s 35ms/step
Accuracy: 0.9466101694915254
Precision: 0.9481945840848515
Recall: 0.9466101694915253
F1-Score: 0.9464724878204224
AUC: 0.9990359438924605
